### Processing

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
path = "/content/drive/My Drive/M2/DeepLearning"

import sys
sys.path.append(path)

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np


#################################################
# Dataset
#################################################
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))
print(train_images.shape)
print(test_images.shape)


import mnist_reader
X_train, y_train = mnist_reader.load_mnist(path+'/Datasets', kind='train')
X_test, y_test = mnist_reader.load_mnist(path+'/Datasets', kind='t10k')
print(X_train.shape)
print(X_test.shape)

X_train, X_test, train_images, test_images = X_train / 255.0, X_test / 255.0, train_images / 255.0, test_images / 255.0
#X_train, X_test, train_images, test_images = X_train / 127.5 - 1, X_test / 127.5 - 1, train_images / 127.5 - 1, test_images / 127.5 - 1


#################################################
# Functions
#################################################
num_examples_to_generate=16
def save_images(predictions, epoch):
  fig = plt.figure(figsize=(4,4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0] * 255.0, cmap='gray')
      #plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')

  #plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  save_images(predictions, epoch)

In [0]:
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score

nmi = normalized_mutual_info_score
ari = adjusted_rand_score

kmeans = KMeans(n_clusters=10, n_init=20)
kmeans.fit(X_train)

print(nmi(kmeans.labels_, y_train))
print(ari(kmeans.labels_, y_train))

### **1. Deep Autoencoder**

In [0]:
from tensorflow import keras
from tensorflow.keras import Input, layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
from scipy import io, sparse
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from statistics import mean, stdev
import os
import pandas as pd

nmi = normalized_mutual_info_score
ari = adjusted_rand_score



#################################################
# Hyper-parameters
#################################################
epoch = 100
batch_size = 256

encoding_dim = 7*7
hidden_dim = 256

#activation='elu'
activation='leaky_relu'
optimizer='adam'
#loss='mean_squared_error'
loss='binary_crossentropy'
#################################################

out_activation = 'sigmoid' if loss=='binary_crossentropy' else 'linear'
activation = None if activation=='leaky_relu' else activation

#################################################
# Auto-encoder
#################################################
input_img = Input(shape=(784,))


####### Encoder #######
#   Layer
encoded = Dense(hidden_dim, activation=activation)(input_img)
if activation is None: encoded = layers.LeakyReLU()(encoded)

#   Layer
"""encoded = Dense(hidden_dim//4, activation=activation)(encoded)
if activation is None: encoded = layers.LeakyReLU()(encoded)"""

#   Layer
encoded = Dense(encoding_dim, activation=out_activation)(encoded)


####### Decoder #######
decoded = encoded
#   Layer
"""decoded = Dense(hidden_dim//4, activation=activation)(decoded)
if activation is None: decoded = layers.LeakyReLU()(decoded)"""

#   Layer
decoded = Dense(hidden_dim, activation=activation)(decoded)
if activation is None: decoded = layers.LeakyReLU()(decoded)

#   Layer
decoded = Dense(784, activation=out_activation)(decoded)




res_file_name = path+"/ae_res_"+str(encoding_dim)+".txt"

best_nmi, best_ari = 0, 0 
# Get old results
if os.path.exists(res_file_name):
    df = pd.read_csv(res_file_name, header=None)
    best_nmi, best_ari = df.iloc[:,0].max(), df.iloc[:,1].max()
    print(best_nmi, " _ ", best_ari)

nmis, aris = [], []
for _ in range(20):

    ####### Make model #######
    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer=optimizer, loss=loss)

    autoencoder.summary()

    ####### Train #######
    history = autoencoder.fit(X_train, X_train,
                    epochs=epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_test, X_test))

    ####### Encode images #######
    encoder = Model(input_img, encoded)
    encoded_images = encoder.predict(X_train)
    print(encoded_images)
    #################################################



    #################################################
    # KMeans
    #################################################
    print("run KMeans")
    kmeans = KMeans(n_clusters=10, n_init=20)
    kmeans.fit(encoded_images)

    res_nmi, res_ari = nmi(kmeans.labels_, y_train), ari(kmeans.labels_, y_train)
    nmis.append(res_nmi)
    aris.append(res_ari)
    print(res_nmi)
    print(res_ari)
    #################################################

    if mean([res_nmi, res_ari]) > mean([best_nmi, best_ari]):
        best_nmi, best_ari = res_nmi, res_ari

        ####### Plot and Save #######
        t = np.arange(0, epoch)
        fig, ax = plt.subplots()
        ax.plot(t, history.history["loss"], label='Train loss')
        ax.plot(t, history.history["val_loss"], label='Test loss')
        legend = ax.legend(loc='upper right', shadow=True)
        ax.set(xlabel='epoch', ylabel='loss')
        ax.grid()
        plt.savefig(path+"/ae_loss_"+str(encoding_dim)+".svg", format="svg")
        plt.show()

        ####### Save encoding #######
        io.savemat(path+"/ae_encoded_"+str(encoding_dim)+".mat", {'X' : sparse.csr_matrix(encoded_images)})

    res_file = open(res_file_name, "a")
    towrite = str(res_nmi) + ", " + str(res_ari) + "\n"
    res_file.write(towrite)
    res_file.close()        


## **2. Deep Convolutional Autoencoder**

### Model

In [0]:
from tensorflow import keras
from tensorflow.keras import Input, layers, models
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
from math import sqrt

nmi = normalized_mutual_info_score
ari = adjusted_rand_score



#################################################
# Hyper-parameters
#################################################
epoch = 10
batch_size = 256

encoding_dim = 7*7

#activation='relu'
activation='leaky_relu'
optimizer='adam'
#loss='mean_squared_error'
loss='binary_crossentropy'
num_conv=3

use_batch_norm_encoded = False
use_batch_norm_decoded = False
#################################################

encoding_size = int(sqrt(encoding_dim))
out_activation = 'sigmoid' if loss=='binary_crossentropy' else 'linear'
activation = None if activation=='leaky_relu' else activation

def conv_layer(input_tensor, filters=32, kernel=(3,3), strides=1, 
               activation="relu", use_batch_norm=False, padding='valid'):
  out = layers.Conv2D(filters, kernel, strides=strides, activation=activation, padding=padding)(input_tensor)
  if use_batch_norm:     out = layers.BatchNormalization()(out)
  if activation is None: out = layers.LeakyReLU()(out)
  return out

def deconv_layer(input_tensor, filters=32, kernel=(3,3), strides=1, 
                 activation="relu", use_batch_norm=False, padding='valid'):
  out = layers.Conv2DTranspose(filters, kernel, strides=strides, activation=activation, padding=padding)(input_tensor)
  if use_batch_norm:     out = layers.BatchNormalization()(out)
  if activation is None: out = layers.LeakyReLU()(out)
  return out

def make_conv_autoencoder_model():
  #################################################
  # Convolutional Auto-encoder
  #################################################
  input_img = Input((28, 28, 1))

  ####### Encoder #######
  encoded = input_img
  #   Layer
  for _ in range(num_conv):
      encoded = conv_layer(encoded, filters=32, kernel=(3,3), strides=1, 
                          activation=activation, use_batch_norm=use_batch_norm_encoded,
                          padding='same')
  encoded = layers.MaxPooling2D((2, 2))(encoded)

  #   Layer
  for _ in range(num_conv):
      encoded = conv_layer(encoded, filters=64, kernel=(3,3), strides=1, 
                          activation=activation, use_batch_norm=use_batch_norm_encoded,
                          padding='same')
  encoded = layers.MaxPooling2D((2, 2))(encoded)

  #   Layer
  for _ in range(num_conv-1):
      encoded = conv_layer(encoded, filters=64, kernel=(3,3), strides=1, 
                          activation=activation, use_batch_norm=use_batch_norm_encoded,
                          padding='same')
  encoded = conv_layer(encoded, filters=64, kernel=(3,3), strides=1, 
                      activation=activation, use_batch_norm=use_batch_norm_encoded)

  #   Layer
  encoded = layers.Flatten()(encoded)
  encoded = layers.Dense(encoding_dim, activation=out_activation)(encoded)

  ####### Decoder #######
  decoded_input = encoded if encoding_size == 7 else layers.Dense(7*7)(encoded)
  decoded = layers.Reshape((7,7,1))(decoded_input)

  #   Layer
  decoded = deconv_layer(decoded, filters=128, kernel=(3,3), strides=2, 
                         activation=activation, use_batch_norm=use_batch_norm_decoded,
                         padding='same')
  for _ in range(num_conv-1):
      decoded = deconv_layer(decoded, filters=128, kernel=(3,3), strides=1, 
                          activation=activation, use_batch_norm=use_batch_norm_decoded,
                          padding='same')

  #   Layer
  decoded = deconv_layer(decoded, filters=64, kernel=(3,3), strides=2, 
                         activation=activation, use_batch_norm=use_batch_norm_decoded,
                         padding='same')  
  for _ in range(num_conv-1):
      decoded = deconv_layer(decoded, filters=64, kernel=(3,3), strides=1, 
                          activation=activation, use_batch_norm=use_batch_norm_decoded,
                          padding='same')

  #   Layer
  for _ in range(num_conv):
      decoded = deconv_layer(decoded, filters=32, kernel=(3,3), strides=1, 
                            activation=activation, use_batch_norm=use_batch_norm_decoded,
                            padding='same')

  #   Layer
  decoded = layers.Conv2D(1, (3, 3), activation=out_activation, padding='same')(decoded)


  ####### Make model #######
  autoencoder = models.Model(input_img, decoded)
  encoder = models.Model(input_img, encoded)
  #decoder = models.Model(encoded, decoded)

  return autoencoder, encoder, decoded

### Training

In [0]:
epoch = 20

import os
import tensorflow as tf
from scipy import io, sparse
from statistics import mean, stdev

best_nmi, best_ari = 0, 0
nmis, aris = [], []

for _ in range(20):
    autoencoder, encoder, decoded = make_conv_autoencoder_model()
    autoencoder.compile(optimizer=optimizer, loss=loss)

    autoencoder.summary()

    """ae_checkpoint_dir = path+'/ae_training_checkpoints'
    ae_checkpoint_prefix = os.path.join(ae_checkpoint_dir, "ckpt")
    ae_checkpoint = tf.train.Checkpoint(autoencoder=autoencoder)

    ae_checkpoint.restore(tf.train.latest_checkpoint(ae_checkpoint_dir))"""

    for _ in range(8):
        generate_and_save_images(autoencoder, 0, 
                              test_input=train_images[np.random.randint(low=0,high=train_images.shape[0],size=16)])

    ####### Train #######
    autoencoder.fit(train_images, train_images,
                  epochs=epoch,
                  batch_size=batch_size,
                  shuffle=True,
                  validation_data=(test_images, test_images))

    #ae_checkpoint.save(file_prefix = ae_checkpoint_prefix)

    for _ in range(8):
        generate_and_save_images(autoencoder, 0, 
                          test_input=train_images[np.random.randint(low=0,high=train_images.shape[0],size=16)])
    #################################################



    #################################################
    # KMeans
    #################################################
    ####### Encode images #######
    encoded_images = encoder.predict(train_images)
    print(encoded_images.shape)

    ####### KMeans #######
    print("run KMeans")
    kmeans = KMeans(n_clusters=10, n_init=20)
    kmeans.fit(encoded_images)

    res_nmi, res_ari = nmi(kmeans.labels_, y_train), ari(kmeans.labels_, y_train)
    nmis.append(res_nmi)
    aris.append(res_ari)
    print(res_nmi)
    print(res_ari)
    #################################################

    """if mean([res_nmi, res_ari]) > mean([best_nmi, best_ari]):
        best_nmi, best_ari = res_nmi, res_ari

        ####### Save encoding #######
        io.savemat(path+"/cae_encoded.mat", {'X' : sparse.csr_matrix(encoded_images)})

    res_file = open(path+"/cae_res.txt", "a")
    towrite = str(res_nmi) + ", " + str(res_ari) + "\n"
    res_file.write(towrite)
    res_file.close()"""





"""# Reconstruct one image per class
index = [2, 17, 6, 4, 20, 9, 19, 7, 24, 1]
to_predict = train_images[index]
predictions = autoencoder.predict(to_predict)    

for i in range(predictions.shape[0]):
    #fig = plt.figure(figsize=(1,2))
    #plt.subplot(1, 2, 1)
    plt.imshow(predictions[i, :, :, 0] * 255.0, cmap='gray')
    plt.axis('off')
    plt.savefig(path+'/image{:d}.svg'.format(i), format="svg")
    #plt.subplot(1, 2, 2)
    plt.imshow(to_predict[i, :, :, 0] * 255.0, cmap='gray')
    plt.axis('off')
    plt.savefig(path+'/image{:d}_original.svg'.format(i), format="svg")"""

## **3. Deep Convolutional Adversarial Autoencoder**

### Model

In [0]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import layers, models, Input
import numpy as np
from statistics import mean

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)



#################################################
# Hyper-parameters
#################################################
batch_size = 256

epoch = 100
noise_dim = 100

n_filter = 128
#################################################



def make_optimizer():
  return tf.keras.optimizers.Adam(1e-4)  

#################################################
# Generator
#################################################
def make_generator_model(input_size=100):
    input_img = Input((input_size, ))
    layer = layers.Dense(7*7*n_filter, use_bias=False)(input_img)
    layer = layers.BatchNormalization()(layer)
    layer = layers.LeakyReLU()(layer)

    layer = layers.Reshape((7, 7, n_filter))(layer)
    #assert model.output_shape == (None, 7, 7, n_filter) # Note: None is the batch size

    """layer = layers.Conv2DTranspose(n_filter, (5, 5), strides=(1, 1), padding='same', use_bias=False)(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.LeakyReLU()(layer)"""

    layer = layers.Conv2DTranspose(n_filter//2, (5, 5), strides=(1, 1), padding='same', use_bias=False)(layer)
    #assert model.output_shape == (None, 7, 7, n_filter//2)(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.LeakyReLU()(layer)

    layer = layers.Conv2DTranspose(n_filter//4, (5, 5), strides=(2, 2), padding='same', use_bias=False)(layer)
    #assert model.output_shape == (None, 14, 14, n_filter//4)(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.LeakyReLU()(layer)

    layer = layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='sigmoid')(layer)
    #assert model.output_shape == (None, 28, 28, 1)

    model = models.Model(input_img, layer)

    return model

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def make_generator_optimizer():
    return make_optimizer()  
#################################################


def make_autoencoder_optimizer():
  return make_optimizer()  

def autoencoder_loss(real_output):
  return cross_entropy(tf.ones_like(real_output), real_output)


#################################################
# Discriminator
#################################################
def make_disc(input_img):
    layer = layers.Conv2D(n_filter//4, (5, 5), strides=(2, 2), padding='same',)(input_img)
    layer = layers.LeakyReLU()(layer)
    #layer = layers.Dropout(0.3)(layer)

    layer = layers.Conv2D(n_filter//2, (5, 5), strides=(2, 2), padding='same')(layer)
    layer = layers.LeakyReLU()(layer)
    #layer = layers.Dropout(0.3)(layer)

    """layer = layers.Conv2D(n_filter, (5, 5), strides=(1, 1), padding='same')(layer)
    layer = layers.LeakyReLU()(layer)
    #layer = layers.Dropout(0.3)(layer)"""

    layer = layers.Flatten()(layer)
    #layer = layers.Dense(256)(layer)    
    layer = layers.Dense(128)(layer)
    layer = layers.Dense(1)(layer)

    model = models.Model(input_img, layer)

    return model

def make_discriminator_model():
    input_img = Input((28, 28, 1))
    return make_disc(input_img)

def make_discriminator_optimizer():
    return make_optimizer()  

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss
#################################################   

def make_gan(generator, discriminator):
	model = models.Sequential()
	
	model.add(generator)
	model.add(discriminator)
	
	opt = make_discriminator_optimizer()
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

def make_aae(autoencoder, discriminator):
	model = models.Sequential()
	
	model.add(autoencoder)
	model.add(discriminator)
	
	opt = make_optimizer()
	#opt = tf.keras.optimizers.Adam(2e-4)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model  
#################################################   


def generate_images(batch_size, noise_dim, generator):
  noise = tf.random.normal([batch_size, noise_dim])
  generated_images = generator(noise, training=True)
  return generated_images
#################################################  




### DCGAN

In [0]:
epoch = 100

import matplotlib.pyplot as plt
import os

discriminator = make_discriminator_model()
discriminator.summary()
discriminator_optimizer = make_discriminator_optimizer()

generator = make_generator_model()
generator.summary()
generator_optimizer =  make_generator_optimizer()

gan_checkpoint_dir = path+'/gan_training_checkpoints'
gan_checkpoint_prefix = os.path.join(gan_checkpoint_dir, "ckpt")
gan_checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

#################################################    
@tf.function
def train_step(images):
    noise = tf.random.normal([batch_size, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

def train(dataset):
  gan_checkpoint.restore(tf.train.latest_checkpoint(gan_checkpoint_dir))
  for ep in range(epoch):
    print("epoch : ",ep, " / ", epoch)
    gen_losses, disc_losses = [], []
    for it in range(int(dataset.shape[0] / batch_size)):
      image_batch = dataset[np.random.randint(low=0,high=dataset.shape[0],size=batch_size)]
      gen_loss, disc_loss = train_step(image_batch)
      gen_losses.append(gen_loss.numpy())
      disc_losses.append(disc_loss.numpy())

    # Save the model
    if (ep + 1) % 10 == 0:
      print("checkpoint...")
      gan_checkpoint.save(file_prefix = gan_checkpoint_prefix)

    generate_and_save_images(generator, ep, test_input=tf.random.normal([num_examples_to_generate, noise_dim]))

    print("Generator loss     =",mean(gen_losses), "\nDiscriminator loss =",mean(disc_losses))

#################################################    
train(train_images)

### DCAA

In [0]:
epoch = 50

import matplotlib.pyplot as plt
import os

print("#################################################\n# DISCRIMINATOR\n#################################################")
discriminator = make_discriminator_model()
#discriminator.compile(optimizer=make_discriminator_optimizer(), loss='binary_crossentropy')
discriminator.summary()
discriminator_optimizer = make_discriminator_optimizer()

print("#################################################\n#GENERATOR\n#################################################")
generator = make_generator_model()
generator.summary()
generator_optimizer =  make_generator_optimizer()

print("#################################################\n# AUTOENCODER\n#################################################")
autoencoder, encoder, decoded = make_conv_autoencoder_model()
autoencoder.compile(optimizer=optimizer, loss='binary_crossentropy')
autoencoder.summary()
autoencoder_optimizer = make_autoencoder_optimizer()

print("#################################################\n# AAE\n#################################################")
aae = make_aae(autoencoder, discriminator)
aae.summary()

"""print("#################################################\n# GAN\n#################################################")
gan = make_gan(generator, discriminator)
gan.summary()"""


gan_checkpoint_dir = path+'/gan_training_checkpoints'
gan_checkpoint_prefix = os.path.join(gan_checkpoint_dir, "ckpt")
gan_checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

ae_checkpoint_dir = path+'/ae_training_checkpoints'
ae_checkpoint_prefix = os.path.join(ae_checkpoint_dir, "ckpt")
ae_checkpoint = tf.train.Checkpoint(autoencoder=autoencoder)


#################################################    
@tf.function
def train_step(images, train_gen=True):
    noise = tf.random.normal([batch_size, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    if train_gen:
      gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
      generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))          

    return gen_loss, disc_loss

def aae_train_step(images):
    with tf.GradientTape() as ae_tape:
      decoded_images = autoencoder(images, training=True)      
      real_output = discriminator(decoded_images, training=True)
      ae_loss = autoencoder_loss(real_output)

    gradients_of_autoencoder = ae_tape.gradient(ae_loss, autoencoder.trainable_variables)    
    autoencoder_optimizer.apply_gradients(zip(gradients_of_autoencoder, autoencoder.trainable_variables))

    return ae_loss

def train(dataset):

  #gan_checkpoint.restore(tf.train.latest_checkpoint(gan_checkpoint_dir))
  ae_checkpoint.restore(tf.train.latest_checkpoint(ae_checkpoint_dir))

  generate_and_save_images(autoencoder, 0, 
                          test_input=dataset[np.random.randint(low=0,high=dataset.shape[0],size=num_examples_to_generate)])
  generate_and_save_images(generator, 0,
                            test_input=tf.random.normal([num_examples_to_generate, noise_dim]))

  # Training pipeline
  for ep in range(epoch):
    print("epoch : ",ep, " / ", epoch)
    losses = {"Autoencoder":[],"GAN":[],"AAE":[],"Generator":[],"Discriminator":[]}

    for it in range(int(dataset.shape[0] / batch_size)):
      image_batch = dataset[np.random.randint(low=0,high=dataset.shape[0],size=batch_size)]

      # Print current losses
      if it % 50 == 0:
        print(it,"/ ",int(dataset.shape[0] / batch_size))
        """for name in losses:
          if losses[name] != []:
            print(name, "loss =", mean(losses[name]))"""

      # GAN training
      gen_loss, disc_loss = train_step(image_batch)
      losses["Generator"].append(gen_loss.numpy())
      losses["Discriminator"].append(disc_loss.numpy())

      # AAE training (without discriminator training)
      aae_loss = aae_train_step(image_batch)
      losses["AAE"].append(aae_loss.numpy())

      # AAE training
      """history = aae.fit(image_batch, np.ones(batch_size),
                epochs=1, verbose=0,
                batch_size=batch_size,
                shuffle=True)
      losses["AAE"].append(history.history['loss'][0])    
      _, _ = train_step(image_batch, train_gen=False)  """

      # Autoencoder training
      history = autoencoder.fit(image_batch, image_batch,
                epochs=1, verbose=0,
                batch_size=batch_size,
                shuffle=True)#,
                #validation_data=(test_images, test_images))
      losses["Autoencoder"].append(history.history['loss'][0])      

    # Plot images
    generate_and_save_images(autoencoder, -1*ep, 
                            test_input=dataset[np.random.randint(low=0,high=dataset.shape[0],size=num_examples_to_generate)])
    generate_and_save_images(generator, ep,
                             test_input=tf.random.normal([num_examples_to_generate, noise_dim]))

    # Print results
    results = autoencoder.evaluate(test_images, test_images, batch_size=batch_size)
    print('Autoencoder validation:', results)    
    for name in losses:
        if losses[name] != []:
          print(name, "loss =", mean(losses[name]))


#################################################    
train(train_images)



#################################################
# KMeans
#################################################
####### Encode images #######
encoded_images = encoder.predict(train_images)
print(encoded_images.shape)

####### KMeans #######
print("run KMeans")
kmeans = KMeans(n_clusters=10, n_init=20)
kmeans.fit(encoded_images)

print(nmi(kmeans.labels_, train_labels))
print(ari(kmeans.labels_, train_labels))
#################################################

io.savemat(path+"/caae2_encoded.mat", {'X' : sparse.csr_matrix(encoded_images)})


""" 
# GAN training      
history = gan.fit(tf.random.normal([batch_size, noise_dim]), np.ones(batch_size),
          epochs=1, verbose=0,
          batch_size=batch_size,
          shuffle=True)
losses["GAN"].append(history.history['loss'][0])

generated_images = generate_images(batch_size, noise_dim, generator)
history = discriminator.fit(generated_images, np.zeros(batch_size),
          epochs=1, verbose=0,
          batch_size=batch_size,
          shuffle=True)
losses["Discriminator"].append(history.history['loss'][0])"""

### DCAA 2

In [0]:
epoch = 50

import matplotlib.pyplot as plt
import os
from scipy import io, sparse

print("#################################################\n# DISCRIMINATOR\n#################################################")
discriminator = make_discriminator_model()
#discriminator.compile(optimizer=make_discriminator_optimizer(), loss='binary_crossentropy')
discriminator.summary()
#discriminator_optimizer = make_discriminator_optimizer()
discriminator_optimizer = tf.keras.optimizers.Adam(1e-5)
disc_lr = 1e-7

"""print("#################################################\n#GENERATOR\n#################################################")
generator = make_generator_model()
generator.summary()"""
#generator_optimizer =  make_generator_optimizer()
generator_optimizer = tf.keras.optimizers.Adam(2e-4)
#generator_optimizer = tf.keras.optimizers.Adam(1e-3)

print("#################################################\n# AUTOENCODER\n#################################################")
autoencoder, encoder, decoded = make_conv_autoencoder_model()
autoencoder.compile(optimizer=optimizer, loss='binary_crossentropy')
autoencoder.summary()
autoencoder_optimizer = make_autoencoder_optimizer()


"""gan_checkpoint_dir = path+'/gan_training_checkpoints'
gan_checkpoint_prefix = os.path.join(gan_checkpoint_dir, "ckpt")
gan_checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)"""

ae_checkpoint_dir = path+'/ae_training_checkpoints'
ae_checkpoint_prefix = os.path.join(ae_checkpoint_dir, "ckpt")
ae_checkpoint = tf.train.Checkpoint(autoencoder=autoencoder)


#################################################    

def train_step(images, train_gen=True):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = autoencoder(images, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    if train_gen:
      gradients_of_generator = gen_tape.gradient(gen_loss, autoencoder.trainable_variables)
      generator_optimizer.apply_gradients(zip(gradients_of_generator, autoencoder.trainable_variables))          

    return gen_loss, disc_loss



def train(dataset):

  #gan_checkpoint.restore(tf.train.latest_checkpoint(gan_checkpoint_dir))
  ae_checkpoint.restore(tf.train.latest_checkpoint(ae_checkpoint_dir))

  generate_and_save_images(autoencoder, 0, 
                          test_input=dataset[np.random.randint(low=0,high=dataset.shape[0],size=num_examples_to_generate)])

  disc_epoch = 3
  for ep in range(disc_epoch):
    print("epoch : ",ep, " / ", disc_epoch)
    losses = {"Autoencoder":[],"GAN":[],"AAE":[],"Generator":[],"Discriminator":[]}

    for it in range(int(dataset.shape[0] / batch_size)):
      image_batch = dataset[np.random.randint(low=0,high=dataset.shape[0],size=batch_size)]
      # GAN pre-training
      gen_loss, disc_loss = train_step(image_batch, train_gen=False)
      losses["Discriminator"].append(disc_loss.numpy())
    print(mean(losses["Discriminator"]))

  global discriminator_optimizer      
  discriminator_optimizer = tf.keras.optimizers.Adam(disc_lr)

  # Training pipeline
  for ep in range(epoch):
    print("epoch : ",ep, " / ", epoch)
    losses = {"Autoencoder":[],"GAN":[],"AAE":[],"Generator":[],"Discriminator":[]}

    for it in range(int(dataset.shape[0] / batch_size)):
      image_batch = dataset[np.random.randint(low=0,high=dataset.shape[0],size=batch_size)]

      # Print current losses
      if it % 50 == 0:
        print(it,"/ ",int(dataset.shape[0] / batch_size))

      # GAN training
      gen_loss, disc_loss = train_step(image_batch)
      losses["Generator"].append(gen_loss.numpy())
      losses["Discriminator"].append(disc_loss.numpy())

      # Autoencoder training
      history = autoencoder.fit(image_batch, image_batch,
                epochs=1, verbose=0,
                batch_size=batch_size,
                shuffle=True)#,
                #validation_data=(test_images, test_images))
      losses["Autoencoder"].append(history.history['loss'][0])     

    # Plot images
    generate_and_save_images(autoencoder, -1*ep, 
                            test_input=dataset[np.random.randint(low=0,high=dataset.shape[0],size=num_examples_to_generate)])

    # Print results
    results = autoencoder.evaluate(test_images, test_images, batch_size=batch_size)
    print('Autoencoder validation:', results)    
    for name in losses:
        if losses[name] != []:
          print(name, "loss =", mean(losses[name]))


#################################################    
train(train_images)



#################################################
# KMeans
#################################################
####### Encode images #######
encoded_images = encoder.predict(train_images)
print(encoded_images.shape)

####### KMeans #######
print("run KMeans")
kmeans = KMeans(n_clusters=10, n_init=20)
kmeans.fit(encoded_images)

print(nmi(kmeans.labels_, train_labels))
print(ari(kmeans.labels_, train_labels))
#################################################

io.savemat(path+"/caae2_encoded.mat", {'X' : sparse.csr_matrix(encoded_images)})

## **4. Ensemble Clustering**

In [0]:
from scipy import io, sparse
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score

nmi = normalized_mutual_info_score
ari = adjusted_rand_score

res_path = path+"/Res/"

ae = io.loadmat(res_path+"ae_49_encoded.mat")['X'].todense()
ae2 = io.loadmat(res_path+"ae_encoded.mat")['X'].todense()
ae3 = io.loadmat(res_path+"ae_100_encoded.mat")['X'].todense()
cae = io.loadmat(res_path+"cae1_encoded.mat")['X'].todense()
cae2 = io.loadmat(res_path+"cae2_encoded.mat")['X'].todense()

data = [ae, ae2, ae3, cae, cae2]

all_data = np.hstack([ae, ae2, ae3, cae, cae2])
print(all_data.shape)

### Ensemble

In [0]:
#####################
# Ensemble
#####################

for _ in range(10):
  kmeans = KMeans(n_clusters=10, n_init=20)
  kmeans.fit(all_data)

  print(nmi(kmeans.labels_, y_train))
  print(ari(kmeans.labels_, y_train))

### Consensus co-assoc and hypergraph

In [0]:
!pip install git+git://github.com/GGiecold/Cluster_Ensembles.git
!apt-get install metis

In [0]:
from statistics import mean, stdev
import numpy as np
#import Cluster_Ensembles as CE
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
from sklearn.cluster import KMeans
import pandas as pd

nmi, ari = normalized_mutual_info_score, adjusted_rand_score

#################################################
# Functions
#################################################
def make_co_assoc(row_labels):
    co_assoc = np.zeros((row_labels.shape[1], row_labels.shape[1]))
    for i in range(row_labels.shape[0]):
        labels = row_labels[i,]
        #temp = np.array([[int(i == j) for i in labels] for j in labels])
        n_values = np.max(labels) + 1
        temp = np.eye(n_values)[labels]
        temp = np.dot(temp, temp.T)
        co_assoc += temp
    return co_assoc


def make_co_assoc_sparse(row_labels):
    co_assoc = sparse.csr_matrix((row_labels.shape[1], row_labels.shape[1]))
    for i in range(row_labels.shape[0]):
        labels = row_labels[i,]
        n_values = np.max(labels) + 1
        temp = sparse.csr_matrix((np.ones(labels.shape), (np.arange(labels.shape[0]), labels)))
        temp = temp.dot(temp.T)
        co_assoc += temp
    return co_assoc    


def run_cluster_ensembles(row_labels, number_of_classes):
    res = CE.cluster_ensembles(cluster_runs=row_labels, N_clusters_max=number_of_classes)
    
    return res

    
def run_co_assoc(row_labels, number_of_classes, use_sparse=False):
    print("make co_assoc...")
    if use_sparse:
        co_assoc = make_co_assoc_sparse(row_labels)
    else:
        co_assoc = make_co_assoc(row_labels)
        
    print("run kMeans...")
    verbose = 1 if use_sparse else 0
    model = KMeans(n_clusters=number_of_classes, n_init=20, verbose=verbose)    
    model.fit(co_assoc)

    return model.labels_
#################################################


t_row_labels = []
it = 5
file_name = path+"/Res/partitions_"+str(it)+".csv"

"""for dataset in data:
    print("==============")
    for i in range(it):
        print(i)
        model = KMeans(n_clusters=10, n_init=20)
        model.fit(dataset)
        t_row_labels.append(model.labels_)
        row_labels = np.array(t_row_labels).T

row_labels = np.array(t_row_labels).T
print(row_labels.shape)
pd.DataFrame(row_labels).to_csv(file_name, index=False)"""

t_row_labels = np.array(pd.read_csv(file_name).T)
t_y_train = y_train

for _ in range(20):
    indexes = []

    for yi in np.unique(y_train):
        t_indexes = np.arange(y_train.shape[0])[y_train == yi]
        indexes.append(t_indexes[np.random.randint(low=0, high=t_indexes.shape[0], size=2000)])

    indexes = np.array(indexes).ravel()
    row_labels = t_row_labels[:,indexes]
    y_train = t_y_train[indexes]

    print(row_labels.shape)
    print(y_train.shape)

    res = run_co_assoc(row_labels, 10)
    #res = run_cluster_ensembles(row_labels, 10)
    """model = KMeans(n_clusters=10, n_init=20)    
    model.fit(row_labels.T)
    res = model.labels_"""

    print(nmi(res, y_train.ravel()), ", ", ari(res, y_train.ravel()))